In [1]:
pip install PyGithub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
Downloads folders from github repo
Requires PyGithub
pip install PyGithub
"""


def get_sha_for_tag(repository: Repository, tag: str) -> str:
    """
    Returns a commit PyGithub object for the specified repository and tag.
    """
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha

    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError("No Tag or Branch exists with that name")
    return matched_tags[0].commit.sha


def download_directory(repository: Repository, sha: str, server_path: str) -> None:
    """
    Download all contents at server_path with commit tag sha in
    the repository.
    """
    if os.path.exists(server_path):
        shutil.rmtree(server_path)

    os.makedirs(server_path)
    contents = repository.get_dir_contents(server_path, ref=sha)

    for content in contents:
        print("Processing %s" % content.path)
        if content.type == "dir":
            os.makedirs(content.path)
            download_directory(repository, sha, content.path)
        else:
            try:
                path = content.path
                file_content = repository.get_contents(path, ref=sha)
                if not isinstance(file_content, ContentFile):
                    raise ValueError("Expected ContentFile")
                file_out = open(content.path, "w+")
                if file_content.content:
                    file_data = base64.b64decode(file_content.content)
                    file_out.write(file_data.decode("utf-8"))
                file_out.close()
            except (GithubException, IOError, ValueError) as exc:
                print("Error processing %s: %s", content.path, exc)


def usage():
    """
    Prints the usage command lines
    """
    print("usage: gh-download --repo=repo --branch=branch --folder=folder")


def main(argv):
    """
    Main function block
    """
    try:
        opts, _ = getopt.getopt(argv, "r:b:f:", ["repo=", "branch=", "folder="])
    except getopt.GetoptError as err:
        print(str(err))
        usage()
        sys.exit(2)
    repo: Optional[str] = None
    branch: Optional[str] = None
    folder: Optional[str] = None
    for opt, arg in opts:
        if opt in ("-r", "--repo"):
            repo = arg
        elif opt in ("-b", "--branch"):
            branch = arg
        elif opt in ("-f", "--folder"):
            folder = arg

    if not repo:
        print("Repo is required")
        usage()
        sys.exit(2)
    if not branch:
        print("Branch is required")
        usage()
        sys.exit(2)
    if not folder:
        print("Folder is required")
        usage()
        sys.exit(2)

    github = Github(None)
    repository = github.get_repo(repo)
    sha = get_sha_for_tag(repository, branch)
    download_directory(repository, sha, folder)
    

In [1]:
import base64
import getopt
import os
import shutil
import sys
from typing import Optional

from github import Github, GithubException
from github.ContentFile import ContentFile
from github.Repository import Repository

In [2]:
g = Github(None)
repo = g.get_repo("retinal-information-processing-lab/Standard_analysis_pipeline")

In [3]:
contents = repo.get_contents("")

In [5]:
for content_file in contents:
    print(content_file)

ContentFile(path="1-Preprocessing.ipynb")
ContentFile(path="2-Analyse_Checkerboard.ipynb")
ContentFile(path="3-Drifting_Gratings.ipynb")
ContentFile(path="4-Chirp+Cell Typing.ipynb")
ContentFile(path="5_Cell_ID_card.ipynb")
ContentFile(path="6-FullField_Flash(Before_&_After_Drug).ipynb")
ContentFile(path="7_Registration_code.ipynb")
ContentFile(path="8_Optogenetic_Holography.ipynb")
ContentFile(path="EulerStim180530.vec")
ContentFile(path="Euler_50Hz_20reps_1024x768pix.vec")
ContentFile(path="README.md")
ContentFile(path="params.py")
ContentFile(path="utils.py")


In [10]:
def download_directory(repository, path):
    g = Github(None)
    repo = g.get_repo("retinal-information-processing-lab/Standard_analysis_pipeline")
    contents = repo.get_contents("")
    for content in contents:
            print("Processing %s" % content.path)
            if content.type == "dir":
                os.makedirs(content.path)
                download_directory(repository, content.path)
            else:
                try:
                    path = content.path
                    file_content = repository.get_contents(path, ref=sha)
                    if not isinstance(file_content, ContentFile):
                        raise ValueError("Expected ContentFile")
                    file_out = open(content.path, "w+")
                    if file_content.content:
                        file_data = base64.b64decode(file_content.content)
                        file_out.write(file_data.decode("utf-8"))
                    file_out.close()
                except (GithubException, IOError, ValueError) as exc:
                    print("Error processing %s: %s", content.path, exc)
